In [1]:
import sys
sys.path.append('/home/quic/QUIC-Projects/')
import numpy as np
import matplotlib.pyplot as plt



from classifiers.quantum.qasvm import PseudoNormQSVM, ParameterArray
from classifiers.quantum.qasvm import QASVM, NormQSVM
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.circuit import ParameterVector
from qiskit.utils import QuantumInstance
from qiskit.providers.aer import AerSimulator
from qiskit_machine_learning.datasets import ad_hoc_data
X, y, Xt, yt = ad_hoc_data(2**4, 2**3, n=2, gap=0.3, plot_data=False, one_hot=False)


feature_map = ZZFeatureMap(2)
var_form = RealAmplitudes(5, reps=1)
quantum_instance = QuantumInstance(AerSimulator(), shots=2**13)
nqsvm = NormQSVM(X, y, quantum_instance, feature_map=feature_map, var_form=var_form, lamda=1e7)
fake_nqsvm = PseudoNormQSVM(X, y, quantum_instance, feature_map=feature_map, var_form=var_form, lamda=1e7)

In [ ]:
from tqdm.notebook import tqdm
real = np.empty(len(Xt))
fake = np.empty(len(Xt))
for i, datum in enumerate(tqdm(Xt)):
    param = np.pi*(2*np.random.rand(nqsvm.num_parameters)-1)
    fake_nqsvm.parameters = param
    nqsvm.parameters = param
    fake[i] = fake_nqsvm.cost_fn(param)
    real[i] = nqsvm.cost_fn(param)

In [6]:
%matplotlib inline
from qiskit.tools.parallel import parallel_map
from qiskit.utils import local_hardware_info
import qiskit.tools.jupyter
from qiskit.circuit import QuantumCircuit
import time
def random_bench(dummy):
    time.sleep(1)
    return None

%qiskit_progress_bar
result = parallel_map(random_bench, range(32), num_processes=local_hardware_info()['cpus'])

KeyboardInterrupt: 

In [ ]:
plt.plot(range(len(Xt)), real, label='real')
plt.plot(range(len(Xt)), fake, label='fake')
plt.legend()
plt.show()

In [2]:
plt.show()

In [3]:
nqsvm.parameters = param
nqsvm.f([Xt[0]])

-0.045410157421875

In [4]:
X, y, Xt, yt = ad_hoc_data(2**4, 2**4, n=2, gap=0.3, plot_data=False, one_hot=False)

In [5]:
feature_map = ZZFeatureMap(2)
var_form = RealAmplitudes(5, reps=5)
quantum_instance = QuantumInstance(AerSimulator(), shots=2**10)
nqsvm = NormQSVM(X, y, quantum_instance, feature_map=feature_map, var_form=var_form)

from classifiers.callback import CostParamStorage
from qiskit.algorithms.optimizers import SPSA
storage = CostParamStorage()
optimizer = SPSA(maxiter=2**10, blocking=True, callback=storage)

In [6]:
optimizer.optimize(None, nqsvm.cost_fn, initial_point=nqsvm.parameters)

KeyboardInterrupt: 

In [ ]:
storage.plot()

In [ ]:
nqsvm.parameters = storage.last_avg(16)
nqsvm.accuracy(Xt, yt)

In [ ]:
feature_map = ZZFeatureMap(2)
var_form = RealAmplitudes(5, reps=5)
quantum_instance = QuantumInstance(AerSimulator(), shots=2**10)
fake_nqsvm = PseudoNormQSVM(X, y, quantum_instance, feature_map=feature_map, var_form=var_form)

from classifiers.callback import CostParamStorage
from qiskit.algorithms.optimizers import SPSA
storage = CostParamStorage()
optimizer = SPSA(maxiter=2**10, blocking=True, callback=storage)

In [ ]:
optimizer.optimize(None, fake_nqsvm.cost_fn, initial_point=fake_nqsvm.parameters)

In [ ]:
storage.plot()

In [ ]:
fake_nqsvm.parameters = storage.last_avg(16)
fake_nqsvm.accuracy(Xt, yt)